In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical


In [17]:
# Load your dataset (adjust the file path and data loading as needed)
data = pd.read_csv('/content/tweet_and_emotion.csv')


In [18]:
# Split the dataset into training and testing sets
X = data['tweet']  # Text data
y = data['emotions']  # Emotion labels

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1104)

In [20]:
# Tokenize the text data
max_words = 10000  # Maximum number of words to keep
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

In [21]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [22]:
# Pad sequences to have a consistent length
max_sequence_length = 100  # Adjust as needed
X_train_padded = pad_sequences(X_train_seq, maxlen=max_sequence_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_sequence_length, padding='post', truncating='post')

In [23]:
emotion_label_mapping = {
    'sadness': 0,
    'joy': 1,
    'anger': 2,
    'fear': 3,
    'neutral': 4
}


In [33]:
y_train_encoded = y_train.map(emotion_label_mapping)
y_test_encoded = y_test.map(emotion_label_mapping)

In [34]:
num_classes = len(emotion_label_mapping)
y_train_encoded = to_categorical(y_train_encoded, num_classes)
y_test_encoded = to_categorical(y_test_encoded, num_classes)

In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Bidirectional
from tensorflow.keras.utils import to_categorical

In [37]:
# Build an LSTM model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_sequence_length))
model.add(SpatialDropout1D(0.2))  # Dropout layer
model.add(Bidirectional(LSTM(64, return_sequences=True)))  # Bidirectional LSTM
model.add(Bidirectional(LSTM(64)))  # Bidirectional LSTM
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


In [40]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [41]:
# Train the model
model.fit(X_train_padded, y_train_encoded, epochs=30, batch_size=64, validation_data=(X_test_padded, y_test_encoded))

Epoch 1/30
83/83 [==============================] - 51s 561ms/step - loss: 1.5782 - accuracy: 0.2620 - val_loss: 1.5430 - val_accuracy: 0.2935
Epoch 2/30
83/83 [==============================] - 28s 342ms/step - loss: 1.3514 - accuracy: 0.4151 - val_loss: 1.2291 - val_accuracy: 0.5267
Epoch 3/30
83/83 [==============================] - 28s 341ms/step - loss: 0.6749 - accuracy: 0.7634 - val_loss: 0.9627 - val_accuracy: 0.6691
Epoch 4/30
83/83 [==============================] - 29s 351ms/step - loss: 0.3180 - accuracy: 0.9032 - val_loss: 1.0644 - val_accuracy: 0.6915
Epoch 5/30
83/83 [==============================] - 28s 342ms/step - loss: 0.1830 - accuracy: 0.9439 - val_loss: 1.1104 - val_accuracy: 0.6977
Epoch 6/30
83/83 [==============================] - 29s 352ms/step - loss: 0.1353 - accuracy: 0.9618 - val_loss: 1.2161 - val_accuracy: 0.6930
Epoch 7/30
83/83 [==============================] - 28s 341ms/step - loss: 0.0939 - accuracy: 0.9710 - val_loss: 1.4289 - val_accuracy: 0.6860

In [42]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test_encoded)
print(f'Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')

166/166 [==============================] - 8s 40ms/step - loss: 1.9930 - accuracy: 0.6672
Loss: 1.9930, Accuracy: 0.6672
